<table>
<tr>
    <td width=25%>
        <img src="images/cytech_logo.png">
    </td>
    <td>
        <center>
            <h1>Deep Learning et Applications</h1>
        </center>
    </td>
    <td width=15%>
        Yann Vernaz 
    </td>
</tr>
</table>

<br/><br/>
<center>
    <a style="font-size: 20pt; font-weight: bold">Lab. 5 - Application : Recherche visuelle</a>
</center>

---

L'objectif de ce <i>Lab.</i> est d'utiliser un modèle pré-entraîné (i.e. `ResNet50`) pour extraire les représentations des images afin de construire un outil pour rechercher des images similaires dans une base d'images.

In [ ]:
%matplotlib inline

import os
import h5py
from zipfile import ZipFile
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

# extraction des données (PASCAL VOC)
if not os.path.exists("images_resize"):
    print('Extraction des images ... dans images_resize/')
    zf = ZipFile('images_pascalVOC.zip')
    zf.extractall('.')

# Modèle pré-entraîné (ResNet50)

In [ ]:
import tensorflow as tf

model = tf.keras.applications.ResNet50(include_top=True, weights='imagenet')

In [ ]:
print(model.summary())

## Classification d'une image 

In [ ]:
import cv2
from matplotlib.pyplot import imread
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.applications.imagenet_utils import decode_predictions

image_path = "images_resize/000007.jpg"

img = imread(image_path)
plt.figure(figsize=(10,10))
plt.imshow(img)

img = cv2.resize(img, (224,224)).astype("float32")
img_batch = preprocess_input(img[np.newaxis]) 

predictions = model.predict(img_batch)
decoded_predictions= decode_predictions(predictions)

for s, predicted_class, score in decoded_predictions[0]:
    print("{0} \t {1}%".format(predicted_class, round(100*score,2)))

# Calcul de la représentation des images

À présent nous allons extraire du réseau la représentation vectorielle des images. Cette représentation correspond à la sortie de la dernière couche du réseau `ResNet50`avant la dernière étape de classification (<i>softmax</i>). 

In [ ]:
input_ = model.layers[0].input
output_ = model.layers[-2].output
base_model = tf.keras.models.Model(input_, output_)

In [ ]:
base_model.summary()

In [ ]:
representation = base_model.predict(img_batch)
print("Dimension de la représentation : {0}".format(representation.shape))
print("representation={0}".format(representation))
print("\nProportion de valeurs à zéro (pas d'activation) : {0}%".format(round(100*np.mean(representation[0]==0),2)))

Une image est donc représentée par un vecteur dense de taille 2048. On peut voir que presque 10% des valeurs sont nulles (pas d'activation).

**NOTE**

Le calcul des représentations de toutes les images peut prendre du temps. Elles sont généralement calculées par lots sur GPU. Nous utiliserons des représentations pré-calculées enregistrées au format h5. Pour les personnes intéressées, cela se fait à l'aide du script `process_images.py`.

In [ ]:
images_paths = ["images_resize/" + path for path in sorted(os.listdir("images_resize/"))]

In [ ]:
# Chargement des représentations pré-calculées
h5f = h5py.File('images_embedding.h5','r')
out_tensors = h5f['img_emb'][:]
h5f.close()

Les représentations sont denses.

<hr style="height:3px;border-top:1px solid #fff" />

> **EXRECICE 1**
>
> - Quelle proportion des représentations sont égales à 0 ?<br/><br/>
>
> - Pourquoi y a-t-il des valeurs nulles ?
>

<hr style="height:3px;border-top:1px solid #fff" />

In [ ]:
## Votre code ici
## ...

# Visualization t-SNE

La méthode t-Distributed Stochastic Neighbor Embedding (t-SNE) [1] est une réduction de dimension non linéaire, dont l’objectif est d’assurer que des points proches dans l’espace de départ gardent des positions proches dans l’espace projeté (2D). Dit autrement, la mesure de distance entre points dans l’espace projecté 2D doit refléter la mesure de distance dans l’espace initial.

[1] Laurens van der Maaten and Geoffrey E. Hinton. Visualizing high-dimensional data using t-sne. Journal of Machine Learning Research, 9:2579–2605, 2008.

In [ ]:
from sklearn.manifold import TSNE

images_embedding_tsne = TSNE(perplexity=30).fit_transform(out_tensors)

In [ ]:
plt.figure(figsize=(10, 10))
plt.title("Visualisation t-SNE")
plt.scatter(images_embedding_tsne[:, 0], images_embedding_tsne[:, 1]);
plt.xticks(()); plt.yticks(());
plt.show()

Ajoutons les vignettes des images originales dans la visualisation `t-SNE`.

In [ ]:
def imscatter(x, y, paths, ax=None, zoom=1, linewidth=0):
    if ax is None:
        ax = plt.gca()
    x, y = np.atleast_1d(x, y)
    artists = []
    for x0, y0, p in zip(x, y, paths):
        try:
            im = imread(p)
        except:
            print(p)
            continue
        im = cv2.resize(im,(224,224))
        im = OffsetImage(im, zoom=zoom)
        ab = AnnotationBbox(im, (x0, y0), xycoords='data',
                            frameon=True, pad=0.1, 
                            bboxprops=dict(edgecolor='red',
                                           linewidth=linewidth))
        artists.append(ax.add_artist(ab))
    ax.update_datalim(np.column_stack([x, y]))
    ax.autoscale()
    return artists

In [ ]:
fig, ax = plt.subplots(figsize=(50, 50))
plt.title("Visualisation t-SNE")
imscatter(images_embedding_tsne[:, 0], images_embedding_tsne[:, 1], paths, zoom=0.5, ax=ax)
plt.savefig('tsne.png')

# Recherche visuelle

Nous allons rechercher les images les plus proches (similaires) en utilisant comme distance la norme $L_2$. 

In [ ]:
# utilitaire pour afficher une image 
def display_image(image):
    plt.figure(figsize=(10,10))
    plt.imshow(imread(image))

<hr style="height:3px;border-top:1px solid #fff" />

> **EXRECICE 2**
>
> Implémentez une fonction qui calcule la distance entre un image et toutes les autres<br/><br/>
>
> Utilisez la fonction `np.linalg.norm`.
>

<hr style="height:3px;border-top:1px solid #fff" />

In [ ]:
def most_similar_images(image_index, top_n=5):
    
    # distances entre les images
    dists = ## Votre code ici
            ## ...
        
    sorted_dists = np.argsort(dists)
    return sorted_dists[:top_n]

In [ ]:
# sanity check
image_index = 57

images_similar = most_similar_images(image_index)

# affichage 
result = [display_image(images_paths[image]) for image in images_similar]

# Classification avec les plus proches voisins (Nearest Neighbors) ?

En utilisant ces représentations, on peut construire un classifieur [Nearest Neighbor] (https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm). Cependant, les représentations apprises sur `ImageNet`, qui sont uniquement des images centrées, contrairement aux images de `PascalVOC` qui sont plus générales.

Nous explorons cette possibilité en calculant l'histogramme des similitudes entre une image.

In [ ]:
# norm L2
out_norms = np.linalg.norm(out_tensors, axis=1, keepdims=True)

# normalisation
normed_out_tensors = out_tensors / out_norms

In [ ]:
image_index = 208

# distances 
dists_to_item = np.linalg.norm(out_tensors - out_tensors[image_index], axis=1)

# cosinus similitude
cos_to_item = np.dot(normed_out_tensors, normed_out_tensors[image_index])

plt.figure(figsize=(10,10))
plt.title("Histogramme des similitudes")
plt.hist(cos_to_item, bins=20)
display_image(images_paths[image_index])

In [ ]:
items = np.where(cos_to_item > 0.44)
print(items)
result = [display_image(paths[s]) for s, _ in zip(items[0], range(10))];

Malheureusement, il n'y a pas de séparation évidente des frontières de classe visible sur l'histogramme des similitudes. Nous avons besoin d'une certaine supervision pour pouvoir classer les images.

<hr style="height:3px;border-top:1px solid #fff" />

> **EXERCICE BONUS**
>
> Avec un ensemble de données étiquetées, même avec très peu d'étiquettes par classe, on pourrait le faire :
> 
> - construire un modèle k-Nearest Neighbor,
> 
> - construire un modèle de classification [SVM](https://scikit-learn.org/stable/modules/svm.html).
> 
> Conclure

<hr style="height:3px;border-top:1px solid #fff" />

In [ ]:
## Votre solution ici
## ...